In [1]:
opts = {
    'published_at_start': 'NOW-2DAYS',
    'published_at_end': 'NOW',
    'language': ['en'],
    'per_page': 5,
    'source_locations_country': ['US'],
    'categories_taxonomy': 'aylien',
    'aql': 'categories: {{taxonomy: aylien AND id:ay.sports.nba}}',
    'not_entities_id' : ['N25240567620063168378490754444048642724'], #exclude sponsored
    'sort_by': 'relevance'
}

In [8]:
import os
from dotenv import load_dotenv
from __future__ import print_function
import time
import aylien_news_api
from aylien_news_api.rest import ApiException
from pprint import pprint

load_dotenv()
API_ID = os.getenv("AYLIEN_API_ID")
API_KEY = os.getenv("AYLIEN_API_KEY")

configuration = aylien_news_api.Configuration()
configuration.api_key['X-AYLIEN-NewsAPI-Application-ID'] = API_ID
configuration.api_key['X-AYLIEN-NewsAPI-Application-Key'] = API_KEY

client = aylien_news_api.ApiClient(configuration)
api_instance = aylien_news_api.DefaultApi(client)

In [2]:


try:
    api_response = api_instance.list_stories(**opts)
    pprint(api_response)
except ApiException as e:
    print("Exception when calling DefaultApi->list_stories: %s\n" % e)

{'next_page_cursor': 'OC4xMzU3MjIsNTMzNzkzOTg1Nw==',
 'published_at_end': datetime.datetime(2023, 2, 24, 21, 2, 25, 802580, tzinfo=tzutc()),
 'published_at_start': datetime.datetime(2023, 2, 22, 21, 2, 25, 802524, tzinfo=tzutc()),
 'stories': [{'author': {'avatar_url': None,
                         'id': 1068186860,
                         'name': 'Noah Moyer'},
              'body': "James Harden's headband has arrived in Philadelphia. "
                      "And it's f*cking beautiful.\n"
                      ' Headband Harden recorded 31 points, 6 threes, 7 '
                      'rebounds and 7 assists in a Sixers comeback win against '
                      'the Grizzlies, who are currently 2nd in the Western '
                      "Conference. Harden's performance marked his fourth game "
                      'of the season with 30+ points.\n'
                      "  I don't care if Harden scored 90 points against the "
                      "Grizzlies, his stat line stil

In [5]:
s1 = api_response.stories[0]

def convert_story(story):
    converted = {
        'text': get_summary(story.body, 8), 
        'entrities': format_entities(story.entities), 
        'links': story.links.permalink,
        'authors': story.author.name,
        'thumbnails':  story.media[0].url if len(story.media) > 0 else None
    }
    
    converted['text'] = clean_text_author(story.author.name, converted['text'])
    converted['entrities'] = clean_ents_author(story.author.name, converted['entrities'])
#     print(converted)
#     converted['text'], converted['entrities'], converted['links'] = , , story.links.permalink
    return converted

# def get_summary(story):
#     return story.body if len(story.body) < 2500 else ", ".join(story.summary.sentences)

def format_entities(ents):
    ents_output = [
        (entity.body.surface_forms[0].text, entity.types[0], entity.overall_frequency)
        for entity in ents
        if len(entity.body.surface_forms) > 0 and len(entity.types) > 0 and isinstance(entity.overall_frequency, (int, float))
    ]
    
    if len(ents_output) > 30:
        ents_output = [item[:2] for item in ents_output if item[2] > 1]
    else:
        ents_output = [item[:2] for item in ents_output]
        
    return ents_output

# ents = format_entities(s1.entities)

def clean_ents_author(author, ents):
    for entity in ents:
        if author in entity:
            ents.remove(entity)    
    return ents

def clean_text_author(author, text):
    while author in text:
        text = text.replace(author, "")
    return text

c1 = convert_story(s1)
c1
# summ

# len(s1_summary), s1_summary, ents

{'text': "Headband Harden recorded 31 points, 6 threes, 7 rebounds and 7 assists in a Sixers comeback win against the Grizzlies, who are currently 2nd in the Western Conference. Harden's performance marked his fourth game of the season with 30+ points. If this is potentially the James Harden that we'll see in the playoffs, then the Sixers should start getting sized for rings ASAP. Diving for loose balls, draining threes over an entire starting lineup, and even getting chippy with Dillon Brooks. Side note: I've decided that Dillon Brooks is a dickhead who would be a fantastic Waffle House manager if he wasn't decent at basketball. Anyways, if I wasn't a Sixers fan, I would be calling up the NBA front office to get Harden's headband tested for PEDs. Instead of heading home and calling it a night, he allegedly went into the bathroom and came out with the infamous white band around his noggin. Apparently he proceeded to throw nearly $4 million in one dollar bills and raged on for the next 

In [281]:

# full_article

In [6]:
opts['story_id'], opts['per_page'], opts['not_id'] = s1.id, 2, [s1.id]
opts = {k: v for k, v in opts.items() if k != 'sort_by'}

# per_page for related doesn't work always returns 10

try:
    related_response = api_instance.list_related_stories_get(**opts)
    pprint(related_response)
except ApiException as e:
    print("Exception when calling DefaultApi->list_stories: %s\n" % e)

{'published_at_end': datetime.datetime(2023, 2, 24, 21, 2, 53, 594956, tzinfo=tzutc()),
 'published_at_start': datetime.datetime(2023, 2, 22, 21, 2, 53, 594918, tzinfo=tzutc()),
 'related_stories': [{'author': {'avatar_url': None,
                                 'id': 3199622,
                                 'name': 'Rich Hofmann'},
                      'body': 'Philadelphia 76ers star Joel Embiid took five '
                              'steps off the charity stripe and looked down at '
                              'the floor after missing a free throw.\n'
                              " Embiid did not take part in Thursday morning's "
                              'shootaround because of an illness. And it '
                              'showed last night.\n'
                              '  The big man is enjoying an elite midrange '
                              'shooting season, but on this night against the '
                              'Memphis Grizzlies , he could not s

In [7]:
full_article = [convert_story(s1)] + [convert_story(item) for item in related_response.related_stories[:2]]

In [8]:
full_article

[{'text': "Headband Harden recorded 31 points, 6 threes, 7 rebounds and 7 assists in a Sixers comeback win against the Grizzlies, who are currently 2nd in the Western Conference. Harden's performance marked his fourth game of the season with 30+ points. If this is potentially the James Harden that we'll see in the playoffs, then the Sixers should start getting sized for rings ASAP. Diving for loose balls, draining threes over an entire starting lineup, and even getting chippy with Dillon Brooks. Side note: I've decided that Dillon Brooks is a dickhead who would be a fantastic Waffle House manager if he wasn't decent at basketball. Anyways, if I wasn't a Sixers fan, I would be calling up the NBA front office to get Harden's headband tested for PEDs. Instead of heading home and calling it a night, he allegedly went into the bathroom and came out with the infamous white band around his noggin. Apparently he proceeded to throw nearly $4 million in one dollar bills and raged on for the next

In [9]:
 combined_story = {
    'text': '',
    'entities': set(),
    'links': set(),
    'authors': set(),
    'thumbnails': set()
}

for d in full_article:
    combined_story['text'] += d['text']
    combined_story['entities'].update(d['entrities'])
    combined_story['links'].add(d['links'])
    combined_story['authors'].add(d['authors'])
    combined_story['thumbnails'].add(d['thumbnails'])

# combined_story['text'] = get_summary(combined_story['text'], 8)
combined_story
# len(combined_story['entities'])

{'text': "Headband Harden recorded 31 points, 6 threes, 7 rebounds and 7 assists in a Sixers comeback win against the Grizzlies, who are currently 2nd in the Western Conference. Harden's performance marked his fourth game of the season with 30+ points. If this is potentially the James Harden that we'll see in the playoffs, then the Sixers should start getting sized for rings ASAP. Diving for loose balls, draining threes over an entire starting lineup, and even getting chippy with Dillon Brooks. Side note: I've decided that Dillon Brooks is a dickhead who would be a fantastic Waffle House manager if he wasn't decent at basketball. Anyways, if I wasn't a Sixers fan, I would be calling up the NBA front office to get Harden's headband tested for PEDs. Instead of heading home and calling it a night, he allegedly went into the bathroom and came out with the infamous white band around his noggin. Apparently he proceeded to throw nearly $4 million in one dollar bills and raged on for the next 

In [9]:
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")
import wandb

run = wandb.init(project='HeywireAI', entity="heywire")
prediction_table = wandb.Table(columns=["original_story", "prompt", "usage", "completion"])

def generate_story(source_links, entities, summary, words = 500):

    mod = "text-davinci-003"
    prompt = f"Generate a story and a title for the story you generate and put the title at the first row. The story should involves the following entities and original example. Please make sure the story you generate is at least {words} words long."

    entity_string = "\n".join([f"- {entity[0]}: {entity[1]}" for entity in entities])
    
    if summary:
        story_prompt = f"{prompt}\nOriginal example: {summary}\n Entities: {entity_string}\n\nBegin story:"
    else:
        story_prompt = f"{prompt}\n{entity_string}\n\nBegin story:"
    
    story_completions = openai.Completion.create(
        model = mod,
        prompt = story_prompt,
        max_tokens=int(4000 - (len(story_prompt) / 3)),
    )
        
    print(story_completions["usage"])
    
    prediction_table.add_data(list(source_links), story_prompt, story_completions['usage'], story_completions['choices'][0]['text'])
    
    return story_completions['choices'][0]['text']

wandb: Currently logged in as: akurokhtin (heywire). Use `wandb login --relogin` to force relogin


In [14]:
aylien_combined_story = generate_story(
    combined_story['links'], 
    combined_story['entities'], 
    combined_story['text']
)
aylien_combined_story

{
  "completion_tokens": 656,
  "prompt_tokens": 1169,
  "total_tokens": 1825
}


"\n\nJames Harden: Headband Prodigy\n\nIt was a game for the ages. The Philadelphia 76ers were facing off against the Memphis Grizzlies, the team who are currently second in the Western Conference and boast one of the NBA's most powerful rosters. The Sixers needed something special from their star player, James Harden, if they were to come out triumphant - and boy, did he deliver.\n\nHarden, who had swapped his signature glasses and signature headband, scored 31 points, with 6 of those being three pointers, as well as 7 rebounds and 7 assists. It was his fourth game of the season with 30 or more points, and it showed the Sixers they might have something special on their hands. As NBA analyst Malika Andrews put it, the basketball gods rewarded Embiid's defensive intensity throughout the game with a highlight-reel block on Morant in crunchtime.\n\nThe Sixers were flying around and it seemed as though everybody had a hand in the victory. Ja Morant seemed unstoppable as he pushed the Grizz

In [15]:
wandb.log({'predictions': prediction_table})
wandb.finish()
# prediction_table
print(aylien_combined_story)



James Harden: Headband Prodigy

It was a game for the ages. The Philadelphia 76ers were facing off against the Memphis Grizzlies, the team who are currently second in the Western Conference and boast one of the NBA's most powerful rosters. The Sixers needed something special from their star player, James Harden, if they were to come out triumphant - and boy, did he deliver.

Harden, who had swapped his signature glasses and signature headband, scored 31 points, with 6 of those being three pointers, as well as 7 rebounds and 7 assists. It was his fourth game of the season with 30 or more points, and it showed the Sixers they might have something special on their hands. As NBA analyst Malika Andrews put it, the basketball gods rewarded Embiid's defensive intensity throughout the game with a highlight-reel block on Morant in crunchtime.

The Sixers were flying around and it seemed as though everybody had a hand in the victory. Ja Morant seemed unstoppable as he pushed the Grizzlies forw

In [2]:
harden1 = "Headband Harden recorded 31 points, 6 threes, 7 rebounds and 7 assists in a Sixers comeback win against the Grizzlies, who are currently 2nd in the Western Conference. Harden's performance marked his fourth game of the season with 30+ points. If this is potentially the James Harden that we'll see in the playoffs, then the Sixers should start getting sized for rings ASAP. Diving for loose balls, draining threes over an entire starting lineup, and even getting chippy with Dillon Brooks. Side note: I've decided that Dillon Brooks is a dickhead who would be a fantastic Waffle House manager if he wasn't decent at basketball. Anyways, if I wasn't a Sixers fan, I would be calling up the NBA front office to get Harden's headband tested for PEDs. Instead of heading home and calling it a night, he allegedly went into the bathroom and came out with the infamous white band around his noggin. Apparently he proceeded to throw nearly $4 million in one dollar bills and raged on for the next week and a half."
harden2 = "Philadelphia 76ers star Joel Embiid took five steps off the charity stripe and looked down at the floor after missing a free throw. The Grizzlies were flying around, and Jaren Jackson Jr. , a front-runner for NBA Defensive Player of the Year, played roamer and helped aggressively off of P.J. The first was the opponent: Memphis, led by Ja Morant 's downhill attacks, leads the NBA in points in the paint. It was as if the basketball gods rewarded Embiid's defensive intensity throughout the game with a highlight-reel block on Morant in crunchtime. Harden scored at a high level against an elite defense: 31 points on 10-of-16 shooting, 6 of 9 from beyond the arc and 5 of 7 from the free-throw line. That felt fitting on a night he passed Allen Iverson for 26th on the NBA's all-time scoring list. That's gonna prepare us for the postseason, so I thought it was good for us.” Rivers had to work hard for the win as well. Harden tracked down the ball, hit the floor and found Embiid on an outlet pass for a fast-break dunk."
harden3 = "The Sixers star has reached out to support student, John Hao -- a fan of Harden's -- who was paralyzed in the shooting James Harden's answer started fittingly Thursday night when he was asked about passing Allen Iverson on the NBA's all-time scoring list. I'm just happy to be in the same conversation as him, and I've got a long way to go.” Harden scored 31 against Memphis and was easily the Sixers' top offensive player on a night where Joel Embiid played tremendous defense but had serious shooting woes. The Sixers have won 16 of their last 20 games heading into a matchup Saturday night against the top-seeded Celtics. We don't want to put ourselves in that situation where we're down double-digit points for basically the entire game. That's always a good sign.” Harden also didn't mind putting aside the specifics of Thursday's game to reflect on another topic. As ESPN's Malika Andrews detailed , Harden recently spoke on FaceTime with John Hao, a Michigan State student. So a couple of people from my team flew up there, were actually in the hospital with him, and brought him some things to kind of cheer him up, make him smile. … I gave him my number so whenever he feels like he needs anything, he can call me and I can check up on him.” While Harden and Iverson are obviously different in plenty of ways, the Sixers' 33-year point guard recognizes that his stardom can be similarly powerful outside of basketball."
hardens = harden1 + harden2 + harden3
hardens_sum_of_sums = get_summary(hardens, 8)
hardens_sum_of_sums
# import requests
  
# url = 'https://api.aylien.com/news/related_stories?story_id=5333218341'
  
# payload = {}
  
# headers = {
#     'X-Application-ID': 'a7509cd8',
#     'X-Application-Key': '60cdd1a760d58d700bf6b8fb98cfb285'
# }
  
# response = requests.request("GET", url, headers=headers, data = payload)
# res = response.json()
# print(len(res['related_stories']))

"Headband Harden recorded 31 points, 6 threes, 7 rebounds and 7 assists in a Sixers comeback win against the Grizzlies, who are currently 2nd in the Western Conference. Side note: I've decided that Dillon Brooks is a dickhead who would be a fantastic Waffle House manager if he wasn't decent at basketball. Anyways, if I wasn't a Sixers fan, I would be calling up the NBA front office to get Harden's headband tested for PEDs. The Grizzlies were flying around, and Jaren Jackson Jr. , a front-runner for NBA Defensive Player of the Year, played roamer and helped aggressively off of P.J. Harden scored at a high level against an elite defense: 31 points on 10-of-16 shooting, 6 of 9 from beyond the arc and 5 of 7 from the free-throw line. I'm just happy to be in the same conversation as him, and I've got a long way to go.” Harden scored 31 against Memphis and was easily the Sixers' top offensive player on a night where Joel Embiid played tremendous defense but had serious shooting woes. As ESPN

In [6]:
harder_sums_of_sums_combined = {
    'text' : "Headband Harden recorded 31 points, 6 threes, 7 rebounds and 7 assists in a Sixers comeback win against the Grizzlies, who are currently 2nd in the Western Conference. Side note: I've decided that Dillon Brooks is a dickhead who would be a fantastic Waffle House manager if he wasn't decent at basketball. Anyways, if I wasn't a Sixers fan, I would be calling up the NBA front office to get Harden's headband tested for PEDs. The Grizzlies were flying around, and Jaren Jackson Jr. , a front-runner for NBA Defensive Player of the Year, played roamer and helped aggressively off of P.J. Harden scored at a high level against an elite defense: 31 points on 10-of-16 shooting, 6 of 9 from beyond the arc and 5 of 7 from the free-throw line. I'm just happy to be in the same conversation as him, and I've got a long way to go.” Harden scored 31 against Memphis and was easily the Sixers' top offensive player on a night where Joel Embiid played tremendous defense but had serious shooting woes. As ESPN's Malika Andrews detailed , Harden recently spoke on FaceTime with John Hao, a Michigan State student. … I gave him my number so whenever he feels like he needs anything, he can call me and I can check up on him.” While Harden and Iverson are obviously different in plenty of ways, the Sixers' 33-year point guard recognizes that his stardom can be similarly powerful outside of basketball.",
    'entities': {
        ('@malika_andrews', 'Organization'),
  ('AI', 'Location'),
  ('Allen Iverson', 'Human'),
  ('Atlanta', 'Company'),
  ('Bill Russell', 'Human'),
  ('Celtics', 'Organization'),
  ('Coincidence', 'Organization'),
  ("De'Anthony Melton", 'Human'),
  ('Dillon Brooks', 'Human'),
  ('Doc Rivers', 'Human'),
  ('Donning', 'Organization'),
  ('ESPN', 'Product_(business)'),
  ('Embiid', 'Human'),
  ('FaceTime', 'Product_(business)'),
  ('Grizzlies', 'Nonprofit_organization'),
  ('Grizzlies', 'Organization'),
  ('Hao', 'Location'),
  ('Harden', 'Human'),
  ('Harden-Embiid', 'Organization'),
  ('Headband Harden', 'Human'),
  ('Ja Morant', 'Human'),
  ('James', 'Human'),
  ('James Harden', 'Human'),
  ('Jaren Jackson Jr.', 'Human'),
  ('Joel', 'Human'),
  ('Joel Embiid', 'Human'),
  ('John Hao', 'Human'),
  ('Malika Andrews', 'Human'),
  ('Maxey', 'Community'),
  ('Memphis', 'Community'),
  ('Memphis', 'Location'),
  ('Memphis Grizzlies', 'Organization'),
  ('Miami', 'Company'),
  ('Michigan State', 'Location'),
  ('NBA', 'Nonprofit_organization'),
  ('NBA', 'Organization'),
  ('Paul Reed', 'Human'),
  ('Philadelphia', 'Community'),
  ('Philadelphia', 'Company'),
  ('Philly', 'Location'),
  ('Sam Hinkie', 'Human'),
  ('Sixers', 'Nonprofit_organization'),
  ('Sixers', 'Organization'),
  ('Things', 'Organization'),
  ('Tobias Harris', 'Human'),
  ('Tucker', 'Human'),
  ('Tyrese Maxey', 'Human'),
  ('USA', 'Country'),
  ('Waffle House', 'Product_(business)'),
  ('Wells Fargo', 'Financial_institution'),
  ('Western Conference', 'Location'),
  ('— Malika Andrews', 'Human')},
 'links': {'https://medium.com/@TheGrannyShot/james-hardens-headband-may-be-tested-for-peds-d890237eeec3',
  'https://theathletic.com/4247773/2023/02/24/joel-embiid-sixers-grizzlies/',
  'https://www.nbcphiladelphia.com/news/sports/sixers-james-harden-aims-to-uplift-shooting-survivor/3507902/'},
 'authors': {'Noah Levick', 'Noah Moyer', 'Rich Hofmann'},
 'thumbnails': {'https://cdn.theathletic.com/app/uploads/2023/02/24081050/USATSI_20070236-1024x683.jpg',
  'https://media.nbcphiladelphia.com/2023/02/James-Harden-Celebration-USATSI.jpg?quality=85&strip=all&resize=1200%2C675',
  'https://miro.medium.com/max/1200/1*8QB75Pg1c_glihW2m8UPMQ.jpeg'}}

harder_sums_of_sums_combined


{'text': "Headband Harden recorded 31 points, 6 threes, 7 rebounds and 7 assists in a Sixers comeback win against the Grizzlies, who are currently 2nd in the Western Conference. Side note: I've decided that Dillon Brooks is a dickhead who would be a fantastic Waffle House manager if he wasn't decent at basketball. Anyways, if I wasn't a Sixers fan, I would be calling up the NBA front office to get Harden's headband tested for PEDs. The Grizzlies were flying around, and Jaren Jackson Jr. , a front-runner for NBA Defensive Player of the Year, played roamer and helped aggressively off of P.J. Harden scored at a high level against an elite defense: 31 points on 10-of-16 shooting, 6 of 9 from beyond the arc and 5 of 7 from the free-throw line. I'm just happy to be in the same conversation as him, and I've got a long way to go.” Harden scored 31 against Memphis and was easily the Sixers' top offensive player on a night where Joel Embiid played tremendous defense but had serious shooting woes

In [10]:
aylien_combined_story_from_sums = generate_story(
    harder_sums_of_sums_combined['links'], 
    harder_sums_of_sums_combined['entities'], 
    harder_sums_of_sums_combined['text']
)
aylien_combined_story_from_sums

{
  "completion_tokens": 1031,
  "prompt_tokens": 719,
  "total_tokens": 1750
}


' \n"The Great Comeback: How Headband Harden And The Sixers Rallied To Take The Win"\n\nMalika Andrews was sitting courtside at Wells Fargo Center, as the Philadelphia 76ers battled against the Memphis Grizzlies. The Sixers started off slowly at the beginning of the game, with Joel Embiid playing tremendous defense yet having serious shooting woes. Tucker and James had been having difficulty getting back on defense, and the Grizzlies were flying around the court. Jaren Jackson Jr. was playing roamer and helping off-ball aggressively off of P.J. Tucker. It seemed inevitable that the Grizzlies would make away with a win.\n\nBut then, something shifted. It was as if a force of nature had propelled its way onto the court, for none other than Headband Harden had stepped up his game. Harden started draining threes and long-range shots, and penetrations to the paint. His flawless combination of skill and technique proved too much even for grizzled veteran Joel, who can be easily considered a 

In [11]:
wandb.log({'predictions': prediction_table})
wandb.finish()
print(aylien_combined_story_from_sums)

 
"The Great Comeback: How Headband Harden And The Sixers Rallied To Take The Win"

Malika Andrews was sitting courtside at Wells Fargo Center, as the Philadelphia 76ers battled against the Memphis Grizzlies. The Sixers started off slowly at the beginning of the game, with Joel Embiid playing tremendous defense yet having serious shooting woes. Tucker and James had been having difficulty getting back on defense, and the Grizzlies were flying around the court. Jaren Jackson Jr. was playing roamer and helping off-ball aggressively off of P.J. Tucker. It seemed inevitable that the Grizzlies would make away with a win.

But then, something shifted. It was as if a force of nature had propelled its way onto the court, for none other than Headband Harden had stepped up his game. Harden started draining threes and long-range shots, and penetrations to the paint. His flawless combination of skill and technique proved too much even for grizzled veteran Joel, who can be easily considered a front-

In [144]:
related_response.related_stories[0]

{'author': {'avatar_url': None, 'id': 3583220, 'name': 'Matt Peralta'},
 'body': "D'Angelo Russell's first stint with the Los Angeles Lakers didn't go "
         'according to plan as the organization soured on him quickly.\n'
         ' Although Russell was clearly a talented player, the organization '
         'had issues with his inconsistent play and his bouts of immaturity. '
         'With a new front office regime led by Magic Johnson, the Lakers '
         'selected Lonzo Ball No. 2 overall in the 2017 NBA Draft. Johnson '
         'cited a need for a leader on the team , a direct shot at Russell who '
         'was eventually traded to the Brooklyn Nets.\n'
         "  Now, Russell's found his way back to Los Angeles and seems ready "
         'for a second chance with the team who initially gave up on him. So '
         'far, the early returns have been fruitful and even Johnson had to '
         "admit that he's seen growth out of Russell, via Bryan Kalbrosky of "
         '

In [152]:
print(related_response.related_stories[0].body)

D'Angelo Russell's first stint with the Los Angeles Lakers didn't go according to plan as the organization soured on him quickly.
 Although Russell was clearly a talented player, the organization had issues with his inconsistent play and his bouts of immaturity. With a new front office regime led by Magic Johnson, the Lakers selected Lonzo Ball No. 2 overall in the 2017 NBA Draft. Johnson cited a need for a leader on the team , a direct shot at Russell who was eventually traded to the Brooklyn Nets.
  Now, Russell's found his way back to Los Angeles and seems ready for a second chance with the team who initially gave up on him. So far, the early returns have been fruitful and even Johnson had to admit that he's seen growth out of Russell, via Bryan Kalbrosky of USA Today
  “I think D'Angelo has really grown and matured. He can score the basketball. He can pass the basketball. He doesn't turn it over a lot. That's what I like about him, too. That pick-and-roll with Anthony Davis is goin

In [148]:
related_response.related_stories[0].id

5333008993

In [1]:
from __future__ import absolute_import
from __future__ import division, print_function, unicode_literals

from sumy.parsers.html import HtmlParser
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer as Summarizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words

def get_summary(article, count):
    LANGUAGE = "english"
    SENTENCES_COUNT = count

    if __name__ == "__main__":
        parser = PlaintextParser.from_string(article, Tokenizer(LANGUAGE))
        stemmer = Stemmer(LANGUAGE)

        summarizer = Summarizer(stemmer)
        summarizer.stop_words = get_stop_words(LANGUAGE)

        summary = " ".join(str(sentence) for sentence in summarizer(parser.document, SENTENCES_COUNT))
        return summary

C:\Users\beaconproduction\sites\local\python\heywire\openai\venv\Lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.14) or chardet (5.1.0)/charset_normalizer (2.0.7) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [160]:
s11_sum = get_summary(related_response.related_stories[0].body, 8)
s11_sum

"D'Angelo Russell's first stint with the Los Angeles Lakers didn't go according to plan as the organization soured on him quickly. Johnson cited a need for a leader on the team , a direct shot at Russell who was eventually traded to the Brooklyn Nets. So far, the early returns have been fruitful and even Johnson had to admit that he's seen growth out of Russell, via Bryan Kalbrosky of USA Today “I think D'Angelo has really grown and matured. You have to pick your poison for what you try to do whether it's Russell coming off the PnR or Anthony Davis rolling to the basket and they'll have some more time playing with LeBron. That's for sure.” There's no denying that on the court Russell has become a much better offensive player and already looks like a perfect fit next to LeBron James and Anthony Davis. Report Ad Off the floor, though, Russell has looked and sounded like a more self-aware person who understands the opportunity in front of him. While it's important for him to figure out hi

In [158]:
import sys
sys.path.append('../api/')
import api

s11_ents = api.text_to_ents(related_response.related_stories[0].body)
s11_ents

[('selge', 'PERSON'),
 ('russell', 'PERSON'),
 ('kneermer', 'PERSON'),
 ('rob pelinka checked', 'PERSON'),
 ('selge bolig', 'PERSON'),
 ('nba draft', 'ORG'),
 ('the brooklyn nets', 'FAC'),
 ('forge of empires', 'ORG'),
 ('2', 'CARDINAL'),
 ('mind darvin', 'PERSON'),
 ('undo\n  anklefit\n  lider', 'ORG'),
 ('forge of empires - free online game', 'PERSON'),
 ('davis', 'PERSON'),
 ('disse', 'GPE'),
 ('caresole\n  ', 'ORG'),
 ('forge of empires - free online game\n  test', 'ORG'),
 ('javascript', 'PERSON'),
 ('søkeannonser', 'PERSON'),
 ("d'angelo", 'PERSON'),
 ('los angeles', 'GPE'),
 ('nesten gitt bort', 'PERSON'),
 ('darvin ham', 'PERSON'),
 ('second', 'ORDINAL'),
 ('norway', 'GPE'),
 ('james', 'PERSON'),
 ('mid-season', 'DATE'),
 ('lakers', 'PERSON'),
 ('lebron', 'PERSON'),
 ('ww iq test\n  sjekk', 'EVENT'),
 ('boligalarmer.no', 'ORG'),
 ('22', 'CARDINAL'),
 ('anthony davis', 'PERSON'),
 ('pnr', 'GPE'),
 ('the new orleans pelicans', 'LOC'),
 ('23', 'CARDINAL'),
 ('youtube', 'ORG'),
 ('